In [26]:
# Restart kernel and reimport everything to ensure fresh imports
import importlib
import sys

# Clear any cached modules
if 'grand_file' in sys.modules:
    del sys.modules['grand_file']
if 'geolib' in sys.modules:
    del sys.modules['geolib']

import ee
import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image, display
import json
sys.path.append('../Setup')  # Go up one level to reach Setup folder
from gee_setup import *

setup_gee()

# Import with fresh reload
from grand_file import snow_difference_map

✅ Matplotlib configured
Google Earth Engine initialized successfully!
EE version: 1.6.3
✅ Matplotlib configured
Google Earth Engine initialized successfully!
EE version: 1.6.3
✅ Matplotlib configured
Google Earth Engine initialized successfully!
EE version: 1.6.3
✅ Matplotlib configured
Google Earth Engine initialized successfully!
EE version: 1.6.3
✅ Matplotlib configured
Google Earth Engine initialized successfully!
EE version: 1.6.3
Google Earth Engine initialized successfully!
EE version: 1.6.3


In [17]:
ee.Authenticate()

True

In [22]:
# Small test area in Colorado Rockies for high-resolution testing
small_aoi = ee.Geometry.Polygon([[
    [-106.8, 39.0],
    [-106.6, 39.0], 
    [-106.6, 39.1],
    [-106.8, 39.1],
    [-106.8, 39.0]
]])

# Test with small area to get 30m resolution
snow_difference_map(small_aoi, 
                   output_filename='small_area_test_30m',
                   cloud_cover=20)

Small area detected (0.0 sq deg), using native 30m resolution
Exported: outputs\small_area_test_30m_historical.tiff (CRS: EPSG:3857, Scale: 30m)
Exported: outputs\small_area_test_30m_historical.tiff (CRS: EPSG:3857, Scale: 30m)
Exported: outputs\small_area_test_30m_recent.tiff (CRS: EPSG:3857, Scale: 30m)
Exported: outputs\small_area_test_30m_recent.tiff (CRS: EPSG:3857, Scale: 30m)
Exported: outputs\small_area_test_30m_difference.tiff (CRS: EPSG:3857, Scale: 30m)
Exported: outputs\small_area_test_30m_difference.tiff (CRS: EPSG:3857, Scale: 30m)
Saved interactive map: outputs\small_area_test_30m.html
Saved interactive map: outputs\small_area_test_30m.html


In [23]:
# Test original Rocky Mountains area with adaptive resolution
rocky_aoi = ee.Geometry.Polygon([[
    [-106.99065650552905, 39.088354116315266],
    [-106.71050513834155, 38.652122067998306], 
    [-106.14470923990405, 38.596331654032184],
    [-106.40838111490405, 39.21189167999414],
    [-106.99065650552905, 39.088354116315266]
]])

# This should use adaptive resolution (likely 120m or 250m)
snow_difference_map(rocky_aoi, 
                   output_filename='rocky_mountains_adaptive', 
                   output_format='tiff',
                   cloud_cover=20)

Small area detected (0.5 sq deg), using native 30m resolution
Export failed for rocky_mountains_adaptive_historical: Total request size (55314240 bytes) must be less than or equal to 50331648 bytes.
Retrying with 60m resolution...
Export failed for rocky_mountains_adaptive_historical: Total request size (55314240 bytes) must be less than or equal to 50331648 bytes.
Retrying with 60m resolution...
Exported (fallback): outputs\rocky_mountains_adaptive_historical.tiff (CRS: EPSG:3857, Scale: 60m)
Export failed for rocky_mountains_adaptive_recent: Total request size (55314240 bytes) must be less than or equal to 50331648 bytes.
Retrying with 60m resolution...
Exported (fallback): outputs\rocky_mountains_adaptive_historical.tiff (CRS: EPSG:3857, Scale: 60m)
Export failed for rocky_mountains_adaptive_recent: Total request size (55314240 bytes) must be less than or equal to 50331648 bytes.
Retrying with 60m resolution...
Exported (fallback): outputs\rocky_mountains_adaptive_recent.tiff (CRS: 

In [25]:
# Define coordinates for a region in Peru/Bolivia Andes
aoi2_coords = [[-73.69599934532533,-12.244453922112815],
               [-74.17939778282533,-14.998901983199628],
               [-68.70820637657533,-16.731947710273676],
               [-67.76338215782533,-13.636366870416724],
               [-73.69599934532533,-12.244453922112815]]

# Convert to Earth Engine Geometry
aoi2 = ee.Geometry.Polygon([aoi2_coords])

# Run analysis with appropriate parameters for Andes region
snow_difference_map(aoi2, 
                   output_filename='andes_snow_difference_map_june', 
                   output_format='html', 
                   month_int=6,  # October for Southern Hemisphere
                   cloud_cover=30)  # Higher cloud cover for mountainous region

Large area detected (28.8 sq deg), using 500m resolution
Interactive map created with all layers (area: 28.8 sq deg)
Saved interactive map: outputs\andes_snow_difference_map_june.html
Interactive map created with all layers (area: 28.8 sq deg)
Saved interactive map: outputs\andes_snow_difference_map_june.html


In [27]:
# Test the cleaned-up code with a small area
test_aoi = ee.Geometry.Polygon([[
    [-106.7, 39.05],
    [-106.6, 39.05], 
    [-106.6, 39.1],
    [-106.7, 39.1],
    [-106.7, 39.05]
]])

# Test both raw and visualized TIFF exports
snow_difference_map(test_aoi, 
                   output_filename='cleaned_code_test',
                   cloud_cover=20)

Area: 0.0 sq deg, Resolution: 30m
Raw data exported: outputs\raw_data\cleaned_code_test_historical_raw.tiff (30m)
Visualized exported: outputs\visualized\cleaned_code_test_historical_visualized.tiff (30m)
Raw data exported: outputs\raw_data\cleaned_code_test_recent_raw.tiff (30m)
Visualized exported: outputs\visualized\cleaned_code_test_recent_visualized.tiff (30m)
Raw data exported: outputs\raw_data\cleaned_code_test_difference_raw.tiff (30m)
Visualized exported: outputs\visualized\cleaned_code_test_difference_visualized.tiff (30m)
Interactive map saved: outputs\cleaned_code_test.html
